<a href="https://colab.research.google.com/github/sutanmuleta/AI-Driven-Early-Detection-of-Alzheimer-s-Disease-MRI-Image-Classification/blob/main/alzhemier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
9# Install required libraries
!pip install torch torchvision transformers diffusers --quiet


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 4.2 MB/s eta 0:00:00


In [ ]:
import torch
from torch import nn, optim
from torchvision import transforms, datasets, models
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import numpy as np
import os


In [ ]:
# Unzip the dataset
!unzip '/content/archive (5).zip' -d '/content/Alzheimer_s Dataset'


Streaming output truncated to the last 5000 lines.
  inflating: /content/Alzheimer_s Dataset/Alzheimer_s Dataset/train/MildDemented/mildDem207.jpg  
  inflating: /content/Alzheimer_s Dataset/Alzheimer_s Dataset/train/MildDemented/mildDem208.jpg  
  inflating: /content/Alzheimer_s Dataset/Alzheimer_s Dataset/train/MildDemented/mildDem209.jpg  
  inflating: /content/Alzheimer_s Dataset/Alzheimer_s Dataset/train/MildDemented/mildDem21.jpg  
  inflating: /content/Alzheimer_s Dataset/Alzheimer_s Dataset/train/MildDemented/mildDem210.jpg  
  inflating: /content/Alzheimer_s Dataset/Alzheimer_s Dataset/train/MildDemented/mildDem211.jpg  
  inflating: /content/Alzheimer_s Dataset/Alzheimer_s Dataset/train/MildDemented/mildDem212.jpg  
  inflating: /content/Alzheimer_s Dataset/Alzheimer_s Dataset/train/MildDemented/mildDem213.jpg  
  inflating: /content/Alzheimer_s Dataset/Alzheimer_s Dataset/train/MildDemented/mildDem214.jpg  
  inflating: /content/Alzheimer_s Dataset/Alzheimer_s Dataset/train/

In [ ]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# Define transformations for the training and test sets
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize images to 224x224 pixels
    transforms.ToTensor(),  # Convert images to PyTorch tensors
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])  # Normalize with ImageNet stats
])

# Define the corrected paths for the train and test datasets
train_data_path = '/content/Alzheimer_s Dataset/Alzheimer_s Dataset/train'
test_data_path = '/content/Alzheimer_s Dataset/Alzheimer_s Dataset/test'

# Load the datasets
train_dataset = datasets.ImageFolder(root=train_data_path, transform=transform)
test_dataset = datasets.ImageFolder(root=test_data_path, transform=transform)

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Print dataset sizes
print(f'Train dataset size: {len(train_dataset)}')
print(f'Test dataset size: {len(test_dataset)}')


Train dataset size: 5121
Test dataset size: 1279


In [ ]:
from torchvision.models import resnet18, ResNet18_Weights

# Load a pre-trained ResNet18 model with the new weights syntax
weights = ResNet18_Weights.DEFAULT  # Use the most up-to-date weights
model = resnet18(weights=weights)


In [ ]:
import torch
from torch import nn, optim
from torchvision import models, transforms, datasets
from torch.utils.data import DataLoader
from torchvision.models import resnet18, ResNet18_Weights

# Check if CUDA is available and use GPU if possible
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define transformations for the training and test sets
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize images to 224x224 pixels
    transforms.ToTensor(),  # Convert images to PyTorch tensors
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])  # Normalize with ImageNet stats
])

# Define the corrected paths for the train and test datasets
train_data_path = '/content/Alzheimer_s Dataset/Alzheimer_s Dataset/train'
test_data_path = '/content/Alzheimer_s Dataset/Alzheimer_s Dataset/test'

# Load the datasets
train_dataset = datasets.ImageFolder(root=train_data_path, transform=transform)
test_dataset = datasets.ImageFolder(root=test_data_path, transform=transform)

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Load a pre-trained ResNet18 model with the new weights syntax
weights = ResNet18_Weights.DEFAULT  # Use the most up-to-date weights
model = resnet18(weights=weights)

# Modify the final layer to match the number of classes (4 in this case)
num_features = model.fc.in_features
model.fc = nn.Linear(num_features, 4)  # 4 classes: Non-Demented, Very Mild Demented, Mild Demented, Moderate Demented

# Move the model to the device
model = model.to(device)

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training function with intermediate outputs
def train_model(model, train_loader, criterion, optimizer, num_epochs=5):  # Train for 5 epochs
    model.train()  # Set model to training mode
    for epoch in range(num_epochs):
        running_loss = 0.0
        for i, (inputs, labels) in enumerate(train_loader):
            inputs, labels = inputs.to(device), labels.to(device)

            # Zero the parameter gradients
            optimizer.zero_grad()

            # Forward pass
            outputs = model(inputs)
            loss = criterion(outputs, labels)

            # Backward pass and optimize
            loss.backward()
            optimizer.step()

            # Accumulate loss
            running_loss += loss.item()

            # Print statistics every 10 batches
            if (i + 1) % 10 == 0:
                print(f"Epoch [{epoch + 1}/{num_epochs}], Batch [{i + 1}/{len(train_loader)}], Loss: {loss.item():.4f}")

        # Print average loss per epoch
        print(f"Epoch [{epoch + 1}/{num_epochs}], Average Loss: {running_loss / len(train_loader):.4f}")

# Train the model on the full dataset
train_model(model, train_loader, criterion, optimizer, num_epochs=5)  # Train for 5 epochs


Epoch [1/5], Batch [10/161], Loss: 1.1516
Epoch [1/5], Batch [20/161], Loss: 0.9210
Epoch [1/5], Batch [30/161], Loss: 0.8967
Epoch [1/5], Batch [40/161], Loss: 0.8761
Epoch [1/5], Batch [50/161], Loss: 0.9169
Epoch [1/5], Batch [60/161], Loss: 0.8063
Epoch [1/5], Batch [70/161], Loss: 0.7581
Epoch [1/5], Batch [80/161], Loss: 0.6215
Epoch [1/5], Batch [90/161], Loss: 0.9816
Epoch [1/5], Batch [100/161], Loss: 0.6066
Epoch [1/5], Batch [110/161], Loss: 0.5403
Epoch [1/5], Batch [120/161], Loss: 0.5759
Epoch [1/5], Batch [130/161], Loss: 0.6597
Epoch [1/5], Batch [140/161], Loss: 0.6056
Epoch [1/5], Batch [150/161], Loss: 0.5417
Epoch [1/5], Batch [160/161], Loss: 0.5698
Epoch [1/5], Average Loss: 0.7918
Epoch [2/5], Batch [10/161], Loss: 0.4111
Epoch [2/5], Batch [20/161], Loss: 0.5413
Epoch [2/5], Batch [30/161], Loss: 0.5703
Epoch [2/5], Batch [40/161], Loss: 0.4500
Epoch [2/5], Batch [50/161], Loss: 0.1928
Epoch [2/5], Batch [60/161], Loss: 0.4435
Epoch [2/5], Batch [70/161], Loss: 

In [ ]:
def evaluate_model(model, test_loader, criterion):
    model.eval()  # Set model to evaluation mode
    test_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():  # Disable gradient calculation for efficiency
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            # Forward pass
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            test_loss += loss.item()

            # Get predictions
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    # Calculate average loss and accuracy
    average_loss = test_loss / len(test_loader)
    accuracy = 100 * correct / total

    print(f"Test Loss: {average_loss:.4f}")
    print(f"Test Accuracy: {accuracy:.2f}%")


evaluate_model(model, test_loader, criterion)


Test Loss: 2.1536
Test Accuracy: 60.52%


In [ ]:
from torchvision import transforms, datasets
from torch.utils.data import DataLoader

# Define transformations for the training set with data augmentation
train_transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize images to 224x224 pixels
    transforms.RandomHorizontalFlip(),  # Randomly flip images horizontally
    transforms.RandomRotation(10),  # Randomly rotate images by up to 10 degrees
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2),  # Random color jitter
    transforms.ToTensor(),  # Convert images to PyTorch tensors
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])  # Normalize with ImageNet stats
])

# Define transformations for the test set
test_transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize images to 224x224 pixels
    transforms.ToTensor(),  # Convert images to PyTorch tensors
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])  # Normalize with ImageNet stats
])

# Load the datasets with the new transformations
train_dataset = datasets.ImageFolder(root=train_data_path, transform=train_transform)
test_dataset = datasets.ImageFolder(root=test_data_path, transform=test_transform)

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)


In [ ]:
# Re-train  model
train_model(model, train_loader, criterion, optimizer, num_epochs=5)


Epoch [1/5], Batch [10/161], Loss: 1.6225
Epoch [1/5], Batch [20/161], Loss: 0.8758
Epoch [1/5], Batch [30/161], Loss: 1.1882
Epoch [1/5], Batch [40/161], Loss: 0.8752
Epoch [1/5], Batch [50/161], Loss: 0.8602
Epoch [1/5], Batch [60/161], Loss: 0.6434
Epoch [1/5], Batch [70/161], Loss: 0.6350
Epoch [1/5], Batch [80/161], Loss: 0.6267
Epoch [1/5], Batch [90/161], Loss: 0.6750
Epoch [1/5], Batch [100/161], Loss: 0.6918
Epoch [1/5], Batch [110/161], Loss: 0.9038
Epoch [1/5], Batch [120/161], Loss: 0.6883
Epoch [1/5], Batch [130/161], Loss: 0.5205
Epoch [1/5], Batch [140/161], Loss: 0.6740
Epoch [1/5], Batch [150/161], Loss: 0.6166
Epoch [1/5], Batch [160/161], Loss: 0.6350
Epoch [1/5], Average Loss: 0.8247
Epoch [2/5], Batch [10/161], Loss: 0.5459
Epoch [2/5], Batch [20/161], Loss: 0.5915
Epoch [2/5], Batch [30/161], Loss: 0.5588
Epoch [2/5], Batch [40/161], Loss: 0.7266
Epoch [2/5], Batch [50/161], Loss: 0.6883
Epoch [2/5], Batch [60/161], Loss: 0.6596
Epoch [2/5], Batch [70/161], Loss: 

In [ ]:

evaluate_model(model, test_loader, criterion)


Test Loss: 0.9633
Test Accuracy: 64.11%


In [ ]:

!ls '/content'


sample_data


In [ ]:
from google.colab import files
uploaded = files.upload()


Saving archive (5).zip to archive (5) (1).zip


In [ ]:
import zipfile


with zipfile.ZipFile('archive (5).zip', 'r') as zip_ref:
    zip_ref.extractall('/content')


In [ ]:

!ls '/content'


'Alzheimer_s Dataset'  'archive (5) (1).zip'  'archive (5).zip'   sample_data


In [ ]:
!ls '/content/Alzheimer_s Dataset'


test  train


In [ ]:
import torch
from torch import nn, optim
from torchvision import transforms, datasets, models
from torch.utils.data import DataLoader


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])


test_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])


train_data_path = '/content/Alzheimer_s Dataset/train'
test_data_path = '/content/Alzheimer_s Dataset/test'


train_dataset = datasets.ImageFolder(root=train_data_path, transform=train_transform)
test_dataset = datasets.ImageFolder(root=test_data_path, transform=test_transform)


train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)


model = models.resnet18(pretrained=True)


num_features = model.fc.in_features
model.fc = nn.Sequential(
    nn.Dropout(0.5),
    nn.Linear(num_features, 4)
)


model = model.to(device)


criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


def train_model(model, train_loader, criterion, optimizer, num_epochs=5):
    model.train()
    for epoch in range(num_epochs):
        running_loss = 0.0
        for i, (inputs, labels) in enumerate(train_loader):
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()

            outputs = model(inputs)
            loss = criterion(outputs, labels)

            loss.backward()
            optimizer.step()

            running_loss += loss.item()

            if (i + 1) % 10 == 0:
                print(f"Epoch [{epoch + 1}/{num_epochs}], Batch [{i + 1}/{len(train_loader)}], Loss: {loss.item():.4f}")

        print(f"Epoch [{epoch + 1}/{num_epochs}], Average Loss: {running_loss / len(train_loader):.4f}")


def evaluate_model(model, test_loader, criterion):
    model.eval()
    test_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)


            outputs = model(inputs)
            loss = criterion(outputs, labels)
            test_loss += loss.item()

            # Get predictions
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()


    average_loss = test_loss / len(test_loader)
    accuracy = 100 * correct / total

    print(f"Test Loss: {average_loss:.4f}")
    print(f"Test Accuracy: {accuracy:.2f}%")


train_model(model, train_loader, criterion, optimizer, num_epochs=5)
evaluate_model(model, test_loader, criterion)


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 91.9MB/s]


Epoch [1/5], Batch [10/161], Loss: 1.0543
Epoch [1/5], Batch [20/161], Loss: 0.9629
Epoch [1/5], Batch [30/161], Loss: 0.8476
Epoch [1/5], Batch [40/161], Loss: 0.8007
Epoch [1/5], Batch [50/161], Loss: 0.8493
Epoch [1/5], Batch [60/161], Loss: 1.1229
Epoch [1/5], Batch [70/161], Loss: 0.9622
Epoch [1/5], Batch [80/161], Loss: 0.8715
Epoch [1/5], Batch [90/161], Loss: 0.7005
Epoch [1/5], Batch [100/161], Loss: 0.9910
Epoch [1/5], Batch [110/161], Loss: 0.7297
Epoch [1/5], Batch [120/161], Loss: 0.8592
Epoch [1/5], Batch [130/161], Loss: 1.2018
Epoch [1/5], Batch [140/161], Loss: 0.9677
Epoch [1/5], Batch [150/161], Loss: 0.8707
Epoch [1/5], Batch [160/161], Loss: 1.0073
Epoch [1/5], Average Loss: 0.9264
Epoch [2/5], Batch [10/161], Loss: 0.9530
Epoch [2/5], Batch [20/161], Loss: 1.1711
Epoch [2/5], Batch [30/161], Loss: 0.9348
Epoch [2/5], Batch [40/161], Loss: 0.8584
Epoch [2/5], Batch [50/161], Loss: 0.8613
Epoch [2/5], Batch [60/161], Loss: 0.6377
Epoch [2/5], Batch [70/161], Loss: 

In [13]:
optimizer = optim.Adam(model.parameters(), lr=0.0005)  # Reduced learning rate from 0.001 to 0.0005


In [14]:
train_model(model, train_loader, criterion, optimizer, num_epochs=10)  # Increased epochs from 5 to 10


Epoch [1/10], Batch [10/161], Loss: 0.4943
Epoch [1/10], Batch [20/161], Loss: 0.3988
Epoch [1/10], Batch [30/161], Loss: 0.3598
Epoch [1/10], Batch [40/161], Loss: 0.4020
Epoch [1/10], Batch [50/161], Loss: 0.3727
Epoch [1/10], Batch [60/161], Loss: 0.4479
Epoch [1/10], Batch [70/161], Loss: 0.3982
Epoch [1/10], Batch [80/161], Loss: 0.5547
Epoch [1/10], Batch [90/161], Loss: 0.3388
Epoch [1/10], Batch [100/161], Loss: 0.2599
Epoch [1/10], Batch [110/161], Loss: 0.3694
Epoch [1/10], Batch [120/161], Loss: 0.2201
Epoch [1/10], Batch [130/161], Loss: 0.4055
Epoch [1/10], Batch [140/161], Loss: 0.2379
Epoch [1/10], Batch [150/161], Loss: 0.2621
Epoch [1/10], Batch [160/161], Loss: 0.2997
Epoch [1/10], Average Loss: 0.4077
Epoch [2/10], Batch [10/161], Loss: 0.4230
Epoch [2/10], Batch [20/161], Loss: 0.2621
Epoch [2/10], Batch [30/161], Loss: 0.3466
Epoch [2/10], Batch [40/161], Loss: 0.3655
Epoch [2/10], Batch [50/161], Loss: 0.4931
Epoch [2/10], Batch [60/161], Loss: 0.3258
Epoch [2/10]

In [15]:

evaluate_model(model, test_loader, criterion)


Test Loss: 1.6597
Test Accuracy: 66.07%
